In [1]:
import numpy as np

In [2]:
def Convolution(Spectrum):
    n = len(Spectrum)
    Cov_Spectrum = []
    for i in range(n-1):
        for j in range(i+1,n):
            if(Spectrum[j]-Spectrum[i]>0):
                Cov_Spectrum.append(Spectrum[j]-Spectrum[i])
    return Cov_Spectrum
            

In [3]:
def Quota(Peptide,mass):
    used = 0
    PeptideList = Peptide.split('-')
    for amino_acid in PeptideList:
        if(mass == int(amino_acid)):
            used += 1
    return Total[mass]-used

In [4]:
def SelectMost(Cov_Spectrum,M):
    Count = {}
    Select_Spectrum = []
    for ele in Cov_Spectrum:
        if ele in Count.keys():
            Count[ele] += 1
        else:
            Count[ele] = 1
    c = 0
    pre = -1
    for mass,count in sorted(Count.items(), key=lambda item: item[1],reverse=True):
        if(c<M or pre == count):
            if(mass>=57 and mass<=200):
                c +=1
                Select_Spectrum += [mass]*count
                pre= count
        
    return np.sort(Select_Spectrum)
        

In [5]:
def LinearSpectrum(Peptide):
    Peptides = Peptide.split('-')
    #Peptides = Peptide
    n = len(Peptides)
    PrefixMass = np.zeros(n+1)
    for i in range(0,n):
        PrefixMass[i+1] = PrefixMass[i] + int(Peptides[i])
    LinearSpectrum = [0]
    for i in range(n):
        for j in range(i+1,n+1):
            #add PrefixMass(j) − PrefixMass(i) to LinearSpectrum
            LinearSpectrum.append(PrefixMass[j]-PrefixMass[i])
            
    return np.sort(LinearSpectrum)

In [6]:
def Mass(Peptide):
    mass = 0
    PeptideList = Peptide.split('-')
    #PeptideList = Peptide
    for amino_acid in PeptideList:
        mass += int(amino_acid)
    return mass

In [7]:
def Expand(CandidatePeptides):
    if (not CandidatePeptides):
        #return [[k] for k in set(AminoAcidMass.values())]
        #return set(str(k) for k in set(AminoAcidMass.values()))
        return [str(k) for k in Alphat]
    NewCandidatePeptides = []
    for Peptide in CandidatePeptides:
        for mass in Alphat:
        #for mass in set(AminoAcidMass.values()):
            if(Quota(Peptide,mass)>0):
            #NewPeptide = Peptide + [mass]
                NewPeptide = Peptide + '-'+str(mass)
            #if not CheckRepeatPeptide(NewPeptide,NewCandidatePeptides):
                NewCandidatePeptides.append(NewPeptide)
    return NewCandidatePeptides

In [8]:
def Score(Peptide,Experimental):
    if(not Peptide):
        return 0
    Theoretical = LinearSpectrum(Peptide)
    Tot = {}
    for mass in Theoretical:
        if mass in Tot.keys():
            Tot[mass] += 1
        else:
            Tot[mass] = 1
    score = 0
    for mass in Experimental:
        if(mass in Tot.keys() and Tot[mass]>0):
            score += 1
            Tot[mass] -= 1
    return score

In [9]:
def Trim(Leaderboard, Spectrum, N):
    n = len(Leaderboard)
    LinearScores = {}
    for Peptide in Leaderboard:
        LinearScores[Peptide] =  Score(Peptide, Spectrum)
    
    count = 0
    NewLeaderBoard = []
    preScore = -1
    for Peptide,score in sorted(LinearScores.items(), key=lambda item: item[1],reverse=True):
        #Peptide = Peptide.strip('][').split(', ') 
        #Peptide = [int(x) for x in Peptide]
        #k = len(Peptide)
        if(preScore == score or count<N):
            NewLeaderBoard.append(Peptide)
        else:
            break
        count += 1
        preScore = score
    return NewLeaderBoard

In [25]:
def LeaderboardCyclopeptideSequencing(Sel_Spectrum,Spectrum, N):
    LeaderPeptide = ''
    Leaderboard  = []
    AllEligiblePep = {}
    while True:
        Leaderboard = Expand(Leaderboard)
        Leaderboard = Trim(Leaderboard, Spectrum, N)
        DiscardBoard = []
        for Peptide in Leaderboard:
            if Mass(Peptide) ==  Spectrum[-1]:
                if Score(Peptide, Spectrum) > Score(LeaderPeptide, Spectrum):
                    AllEligiblePep[Peptide] = Score(Peptide, Spectrum)
                    LeaderPeptide = Peptide
                #DiscardBoard.append(Peptide)
            elif Mass(Peptide) > Spectrum[-1]:
                DiscardBoard.append(Peptide)
        Leaderboard = [x for x in Leaderboard if x not in DiscardBoard]
        if not Leaderboard:
            return AllEligiblePep

In [26]:
N = 0
Spectrum = []
M=0
file = open('dataset_369297_8.txt', 'r') 
for i, line in enumerate(file):
    line=line.rstrip('\n')
    if(i==0):
        N = int(line)
    elif(i==1):
        M = int(line)
    else:
        Spectrum = np.array(line.split(' ')).astype(int)


In [27]:
Spectrum = np.sort(Spectrum)
Cov_Spectrum = Convolution(Spectrum)
Sel_Spectrum = SelectMost(Cov_Spectrum,M)
Alphat = set(Sel_Spectrum)
Sel_Spectrum = np.sort(Sel_Spectrum)
Total = {}
for mass in Cov_Spectrum:
    if mass in Total.keys():
        Total[mass] += 1
    else:
        Total[mass] = 1
LeaderPeptide = ''

In [28]:
AllPep = LeaderboardCyclopeptideSequencing(Sel_Spectrum, Spectrum,N)

In [29]:
for pep,score in AllPep.items():
    print(pep,score)

113-128-99-163-128-114-147-186-97-147 54
147-113-128-99-163-128-114-147-97-89-97 56


In [15]:
i = 0
for pep,score in sorted(AllPep.items(), key=lambda item: item[1],reverse=True):
    if score>=50:
        print(pep,end=' ')
    i += 1

147-113-128-99-163-128-114-147-97-89-97 113-128-99-163-128-114-147-186-97-147 97-186-147-114-128-163-99-128-113-147 147-113-128-99-163-128-114-147-186-97 186-147-114-128-163-99-128-113-147-97 128-99-163-128-114-147-186-97-147-113 97-147-113-128-99-163-128-114-147-186 99-163-128-114-147-186-97-147-113-128 163-128-114-147-186-97-147-113-128-99 113-128-99-163-128-114-186-147-97-147 

In [24]:
Score('97-147-113-128-99-163-128-57-57-147-57-129',Spectrum)

54

In [48]:
N = 0
Spectrum = []
M=34
file = open('dataset_369296_2.txt', 'r') 
for i, line in enumerate(file):
    line=line.rstrip('\n')
    if(i==0):
        N = int(line)
    else:
        Spectrum = np.array(line.split()).astype(int)

In [50]:
Spectrum = np.sort(Spectrum)
Cov_Spectrum = Convolution(Spectrum)
Sel_Spectrum = SelectMost(Cov_Spectrum,M)
Alphat = set(Sel_Spectrum)
Sel_Spectrum = np.sort(Sel_Spectrum)
Total = {}
for mass in Cov_Spectrum:
    if mass in Total.keys():
        Total[mass] += 1
    else:
        Total[mass] = 1
LeaderPeptide = ''

In [51]:
AllPep = LeaderboardCyclopeptideSequencing(Sel_Spectrum, Spectrum,N)

In [52]:
for pep,score in AllPep.items():
    print(pep,score)

128-114-147-186-97-147-146-194-163 40
128-99-163-128-114-147-186-97-147-113 54
128-99-163-128-114-147-96-90-97-147-113 57
128-99-163-128-114-147-96-90-97-57-90-113 60
